In [1]:
import pandas as pd
import spotipy
import spotipy.util as util
import sys
sys.path.append('data/')
from spotify_client_details import *

In [2]:
spotify_raw = pd.read_json('data/StreamingHistory0.json')

In [3]:
songs = spotify_raw[['artistName','trackName']].drop_duplicates()
songs1 = songs.head(50)

In [4]:
from spotipy.oauth2 import SpotifyOAuth
scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=CLIENT_ID, 
                                               client_secret=CLIENT_SECRET, redirect_uri='http://localhost/'))


In [5]:
%time
artist = 'Tame Impala'
track = "Solitude is Bliss"

a = sp.search(q=artist + ' ' + track)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.2 µs


In [6]:
print(a['tracks']['items'][0],'\n')
print(a['tracks']['items'][0]['album'].keys())
print(a['tracks']['items'][0]['album']['album_type'])

{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/5INjqkS1o8h1imAzPqGZBb'}, 'href': 'https://api.spotify.com/v1/artists/5INjqkS1o8h1imAzPqGZBb', 'id': '5INjqkS1o8h1imAzPqGZBb', 'name': 'Tame Impala', 'type': 'artist', 'uri': 'spotify:artist:5INjqkS1o8h1imAzPqGZBb'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MG', 'MH', 'MK', 'ML', 'MN', 'MO', 'MR', 'MT

In [9]:
def albumName(artist,track):
    song = sp.search(q='artist:{} track:{}'.format(artist, track))
    if song['tracks']['items']==[]:
        song = sp.search(q=artist + ' ' + track)
    song = song['tracks']['items']
    if song==[]:
        return None, None, None, None, None, None
    else:
        return song[0]['album']['name'], song[0]['duration_ms'], song[0]['popularity'], song[0]['track_number'], song[0]['album']['total_tracks'], song[0]['album']['release_date']
    

In [10]:
songs[['albumName','duration_ms','popularity','track_no','tracks_in_album','album_release_date']] = \
%time songs.apply(lambda row: albumName(row.artistName, row.trackName), axis=1,  result_type="expand")

CPU times: user 8.09 s, sys: 1.64 s, total: 9.73 s
Wall time: 9min 28s


In [11]:
spotify_full = spotify_raw.merge(songs,on=['artistName','trackName'])

In [12]:
spotify_full.to_csv('data/song_features.csv')